In [1]:
import cv2
import mediapipe as mp
import pyautogui
import math

In [5]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# get the width and height of the screen
screen_wid, screen_hei = pyautogui.size()

# Helper function to calculate the distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

clicking = False 

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    else:
        frame = cv2.resize(frame, (1100, 700))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame)
        h, w, _ = frame.shape
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                thump_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                
                finger_x = int(index_finger_tip.x * w)
                finger_y = int(index_finger_tip.y * h)
                
                thump_x = int(thump_tip.x * w)
                thump_y = int(thump_tip.y * h)
                
                screen_x = int(index_finger_tip.x * screen_wid) + 10
                screen_y = int(index_finger_tip.y * screen_hei) + 10
                
                pyautogui.moveTo(screen_x, screen_y)
                
                
                distance = calculate_distance(finger_x, finger_y, thump_x, thump_y)
                
                if distance < 30:
                    if not clicking:
                        pyautogui.click()
                        clicking = True
                    else:
                        clicking = False
                        
                        
                # Visualize the points for index finger and thumb
                cv2.circle(frame, (finger_x, finger_y), 10, (0, 255, 0), cv2.FILLED)  # Index finger
                cv2.circle(frame, (thump_x, thump_y), 10, (0, 0, 255), cv2.FILLED)  # Thumb
                
        cv2.imshow("Hand Gesture Mouse Control with Click", frame)
        if cv2.waitKey(1) == 27:
            break
        
cap.release()
cv2.destroyAllWindows()